<a href="https://colab.research.google.com/github/SKawsar/game_python/blob/main/testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -----------------------------------------------DT-----------------------------------------------------
# hyperparameter tuning Decision Tree Regressor
# criterion_DT will be checkboxes
# criterion_DT = ["gini", "entropy", "log_loss"]
# default: criterion_DT = ["gini"]
criterion_DT = ["gini"]

# max_features_DT will be checkboxes
# max_features_DT = [None, "sqrt", "log2"]
# default: max_features_DT = [None]
max_features_DT = [None]

# max_depth_DT_select can be "list" or "range" (radio button)
# default: max_depth_DT_select = "list"
max_depth_DT_select = "list"  # options are "list" or "range"

if max_depth_DT_select == "list":
    # if user selects "list", then show a textbox
    # default: max_depth_DT = [None]
    max_depth_DT = [None]
else:  # "range"
    # if user selects "range", then show three text boxes (integer only)
    # default: max_depth_DT_start = 3
    # default: max_depth_DT_end = 4
    # default: max_depth_DT_increment = 1
    max_depth_DT_start = 3  # max_depth_DT_start must be greater than 2
    max_depth_DT_end = 4
    max_depth_DT_increment = 1  # max_depth_DT_increment must be greater than 0
    max_depth_DT = [i for i in range(max_depth_DT_start,
                                     max_depth_DT_end,
                                     max_depth_DT_increment)]

min_samples_leaf_DT_select = "list"  # options are "list" or "range"
# default: min_samples_leaf_DT_select = "list"
if min_samples_leaf_DT_select == "list":
    # default: min_samples_leaf_DT = [1]
    min_samples_leaf_DT = [1]  # min_samples_leaf_DT must be greater than 1. Integers only
else:  # "range"
    # default: min_samples_leaf_DT_start = 1
    # default: min_samples_leaf_DT_end = 2
    # default: min_samples_leaf_DT_increment = 1
    min_samples_leaf_DT_start = 1  # min_samples_leaf_DT_start must be greater than 0
    min_samples_leaf_DT_end = 2
    min_samples_leaf_DT_increment = 1  # min_samples_leaf_DT_increment must be greater than 0
    min_samples_leaf_DT = [i for i in range(min_samples_leaf_DT_start,
                                            min_samples_leaf_DT_end,
                                            min_samples_leaf_DT_increment)]

# min_samples_split_DT_select options are "list" or "range" (radio button)
# default: min_samples_split_DT_select = "list"
min_samples_split_DT_select = "list"
if min_samples_split_DT_select == "list":
    # min_samples_split_DT will never contain None
    # default: min_samples_split_DT = [2]
    min_samples_split_DT = [2]
else:
    # default: min_samples_split_DT_start = 2, must be greater than 1
    # default: min_samples_split_DT_end = 3
    # default: min_samples_split_DT_increment = 1, must be greater than 0
    min_samples_split_DT_start = 2
    min_samples_split_DT_end = 3
    min_samples_split_DT_increment = 1
    min_samples_split_DT = [i for i in range(min_samples_split_DT_start,
                                             min_samples_split_DT_end,
                                             min_samples_split_DT_increment)]

parameters_DT = {'criterion': criterion_DT,
                 'max_features': max_features_DT,
                 'max_depth': max_depth_DT,
                 'min_samples_leaf': min_samples_leaf_DT,
                 'min_samples_split': min_samples_split_DT}

# -----------------------------------------------RF-----------------------------------------------------
# hyperparameter tuning Random Forest Regressor
# n_estimators_RF_select can be "list" or "range" (radio button)
# default: n_estimators_RF_select = "list"
n_estimators_RF_select = "list"
if n_estimators_RF_select == "list":
    # if user selects "list", then show a textbox
    # default: n_estimators_RF = [100]
    n_estimators_RF = [100]
else:  # "range"
    # if user selects "range", then show three text boxes
    # default: n_estimators_RF_start = 100
    # default: n_estimators_RF_end = 200
    # default: n_estimators_RF_increment = 100
    n_estimators_RF_start = 100  # n_estimators_RF_start must be greater than 1
    n_estimators_RF_end = 200
    n_estimators_RF_increment = 100  # n_estimators_RF_increment must be greater than zero
    n_estimators_RF = [i for i in range(n_estimators_RF_start,
                                        n_estimators_RF_end,
                                        n_estimators_RF_increment)]

# criterion_RF will be checkboxes
# criterion_RF available options are ["gini", "entropy", "log_loss"]
# default: criterion_RF = ["gini"]
# user must select at least one checkbox
criterion_RF = ["gini"]

# max_features_RF will be checkboxes
# max_features_RF available options [None, "sqrt", "log2"]
# default: max_features_RF = [None]
# user must select at least one checkbox
max_features_RF = ["sqrt"]

# bootstrap_RF will be radio buttons
# bootstrap_RF available options [True, False]
# default: bootstrap_RF = [True]
# user must select at least one radio button
bootstrap_RF = [True]

# max_depth_RF_select can be "list" or "range" (radio button)
# default: max_depth_RF_select = "list"
max_depth_RF_select = "list"  # options are "list" or "range"

if max_depth_RF_select == "list":
    # if user selects "list", then show a textbox
    # default: max_depth_RF = [None]
    max_depth_RF = [None]  # max_depth_Rf must be greater than 2, or None
else:  # "range"
    # if user selects "range", then show three text boxes
    # default: max_depth_RF_start = 3
    # default: max_depth_RF_end = 4
    # default: max_depth_RF_increment = 1
    max_depth_RF_start = 3  # max_depth_RF_start must be greater than 2
    max_depth_RF_end = 4
    max_depth_RF_increment = 1  # max_depth_RF_increment must be greater than zero and integer only
    max_depth_RF = [i for i in range(max_depth_RF_start,
                                     max_depth_RF_end,
                                     max_depth_RF_increment)]

parameters_RF = {'criterion': criterion_RF,
                 'max_features': max_features_RF,
                 'max_depth': max_depth_RF,
                 'bootstrap': bootstrap_RF,
                 'n_estimators': n_estimators_RF}

# -----------------------------------------------XGB-----------------------------------------------------
# hyperparameter tuning XGBoost Regressor
# n_estimators_XGB_select can be "list" or "range" (radio button)
# default: n_estimators_XGB_select = "list"
n_estimators_XGB_select = "list"
if n_estimators_XGB_select == "list":
    # if user selects "list", then show a textbox
    # default: n_estimators_XGB = [100]
    n_estimators_XGB = [100]
else:  # "range"
    # if user selects "range", then show three text boxes
    # default: n_estimators_XGB_start = 100
    # default: n_estimators_XGB_end = 150
    # default: n_estimators_XGB_increment = 50
    n_estimators_XGB_start = 100  # n_estimators_XGB_start must be greater than 1
    n_estimators_XGB_end = 150
    n_estimators_XGB_increment = 50  # n_estimators_XGB_increment must be greater than zero and integer only
    n_estimators_XGB = [i for i in range(n_estimators_XGB_start,
                                         n_estimators_XGB_end,
                                         n_estimators_XGB_increment)]

# max_depth_XGB_select can be "list" or "range" (radio button)
# default: max_depth_XGB_select = "list"
max_depth_XGB_select = "list"  # options are "list" or "range"

if max_depth_XGB_select == "list":
    # if user selects "list", then show a textbox
    # default: max_depth_XGB = [6]
    max_depth_XGB = [6]  # max_depth_XGB must be greater than 2
else:  # "range"
    # if user selects "range", then show three text boxes
    # default: max_depth_XGB_start = 6
    # default: max_depth_XGB_end = 7
    # default: max_depth_XGB_increment = 1
    max_depth_XGB_start = 6  # max_depth_XGB_start must be greater than 2
    max_depth_XGB_end = 7
    max_depth_XGB_increment = 1  # max_depth_XGB_increment must be greater than zero and integer only
    max_depth_XGB = [i for i in range(max_depth_XGB_start,
                                      max_depth_XGB_end,
                                      max_depth_XGB_increment)]

# learning_rate_XGB_select can be "list" or "range" (radio button)
# default: learning_rate_XGB_select = "list"
learning_rate_XGB_select = "list"  # options are "list" or "range"

if learning_rate_XGB_select == "list":
    # if user selects "list", then show a textbox
    # learning_rate_XGB must be within 0.1 to 1
    # default: learning_rate_XGB = [0.3]
    learning_rate_XGB = [0.3]
else:  # "range"
    # if user selects "range", then show three text boxes
    # learning_rate_XGB must be within 0.1 to 1
    # default: learning_rate_XGB_start = 0.3
    # default: learning_rate_XGB_end = 0.4
    # default: learning_rate_XGB_increment = 0.1
    learning_rate_XGB_start = 0.3
    learning_rate_XGB_end = 0.4
    learning_rate_XGB_increment = 0.1  # learning_rate_XGB_increment must be greater than zero
    learning_rate_XGB = [i for i in range(learning_rate_XGB_start,
                                          learning_rate_XGB_end,
                                          learning_rate_XGB_increment)]

parameters_XGB = {'learning_rate': learning_rate_XGB,
                  'max_depth': max_depth_XGB,
                  'n_estimators': n_estimators_XGB}

# -----------------------------------------------MLP-----------------------------------------------------
# hyperparameter tuning Neural Network Regressor
# hidden_layer_sizes_XGB must be tuple
# default: hidden_layer_sizes_MLP = [(100,)]
hidden_layer_sizes_MLP = [(100,)]

# activation_MLP will be checkboxes
# activation_MLP = ["relu", "tanh", "logistic"]
# default: activation_MLP = ["relu"]
activation_MLP = ["relu"]

# solver_MLP will be checkboxes
# solver_MLP = ["sgd", "adam"]
# default: solver_MLP = ["adam"]
solver_MLP = ["adam"]

# learning_rate_init_MLP_select can be "list" or "range" (radio button)
# default: learning_rate_init_MLP_select = "list"
learning_rate_init_MLP_select = "list"  # options are "list" or "range"

if learning_rate_init_MLP_select == "list":
    # if user selects "list", then show a textbox
    # learning_rate_init_MLP must be within 0.001 to 1
    learning_rate_init_MLP = [0.001]
else:  # "range"
    # if user selects "range", then show three text boxes
    # learning_rate_init_MLP must be within 0.001 to 1
    # learning_rate_init_MLP_increment (float) must be greater than zero
    learning_rate_init_MLP_start = 0.001
    learning_rate_init_MLP_end = 0.002
    learning_rate_init_MLP_increment = 0.001
    learning_rate_init_MLP = [i for i in range(learning_rate_init_MLP_start,
                                               learning_rate_init_MLP_end,
                                               learning_rate_init_MLP_increment)]

# max_iter_MLP_select can be "list" or "range" (radio button)
# default: max_iter_MLP_select = "list"
max_iter_MLP_select = "list"  # options are "list" or "range"

if max_iter_MLP_select == "list":
    # if user selects "list", then show a textbox
    # max_iter_MLP must be greater than or equal 200
    max_iter_MLP = [200]
else:  # "range"
    # if user selects "range", then show three text boxes
    # max_iter_MLP must be greater than or equal 200
    max_iter_MLP_start = 200
    max_iter_MLP_end = 300
    max_iter_MLP_increment = 100  # max_iter_MLP_increment must be greater than zero
    max_iter_MLP = [i for i in range(max_iter_MLP_start,
                                     max_iter_MLP_end,
                                     max_iter_MLP_increment)]

parameters_MLP = {"hidden_layer_sizes": hidden_layer_sizes_MLP,
                  "activation": activation_MLP,
                  "solver": solver_MLP,
                  "max_iter": max_iter_MLP,
                  "learning_rate_init": learning_rate_init_MLP}

# -----------------------------------------------LR-----------------------------------------------------
penalty_LR = ["l2"]
c_LR = [1]
solver_LR = ["saga"]
max_iter_LR = [100]
parameters_LR = {'penalty': penalty_LR,
                 'C': c_LR,
                 'solver': solver_LR,
                 'max_iter': max_iter_LR}

# -----------------------------------------------SV-----------------------------------------------------
kernel_SV = ["rbf"]
c_SV = [1]
parameters_SV = {"kernel": kernel_SV,
                 "c": c_SV}

# -----------------------------------------------KNN-----------------------------------------------------
n_neighbors_KNN = [5]
parameters_KNN = {'n_neighbors': n_neighbors_KNN}

hp = {"DT": parameters_DT,
      "RF": parameters_RF,
      "XGB": parameters_XGB,
      "MLP": parameters_MLP,
      "LR": parameters_LR,
      "SV": parameters_SV,
      "KNN": parameters_KNN}
